# Resources

1. need to see this [mbert](https://huggingface.co/sagorsarker/mbert-bengali-ner) for getting idea about architecture [mbert full code](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/token_classification.ipynb)

### alternate implementation

https://saturncloud.io/blog/implementing-huggingface-bert-using-tensorflow-for-sentence-classification/

### prev codes

1. https://colab.research.google.com/drive/1RjVnXwyBlwGjMozF8is_Yqxw06pbCdNi?usp=sharing

2. https://colab.research.google.com/drive/1v9gQS2evy-KoFN5PPxlRim7bmXiJhLK5?usp=sharing

3. https://colab.research.google.com/drive/16qmrSRdDNnW53trCBk1c19L3rrAzuMtB?usp=sharing

4. https://colab.research.google.com/drive/1pS9azT0kPm5uAPhFXZ9WPRNcngYOk0MW?usp=sharing

5. https://colab.research.google.com/drive/1VOQkncOEtPkj91TbGl9DIFScXOs8lguG?usp=sharing

# Token classification (TensorFlow)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs
# for kaggle
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-qwxrkypl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-qwxrkypl
  Resolved https://github.com/huggingface/transformers.git to commit 35eac0df75c692c5b93c12f7eaf3279cab8bd7ce
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7308510 sha256=030e96cf837814df763ddfaccc959481d181f7156f138e25d9f51ad26f160e0b
  Stored in directory: /tmp/pip-ephem-wheel-cache-p2_77kg7/wheels/

You will need to setup git, adapt your email and name in the following cell.

In [2]:
!git config --global user.email "imtiazahmed5573@gmail.com"
!git config --global user.name "imtiaz114"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

* huggingace prob in [kaggle](https://stackoverflow.com/questions/76363436/cannot-import-name-partialstate-from-accelerate-when-using-huggingface-pipel) solve

In [3]:
from huggingface_hub import notebook_login

notebook_login()

# Dataset

In [4]:
import pandas as pd
dataset_path = "/kaggle/input/hishab-dataset-person-extractor/concatened-augmented-ner-bangla-ds/concatened-augmented-ner-bangla-ds.csv"
df = pd.read_csv(dataset_path,index_col=False)
df.head(5)

,tokens,tags,flag
0,"['\\ufeffউন্নয়নের', 'বিস্ময়', 'বাংলাদেশ', 'বাং...","['O', 'O', 'B-LOC', 'B-LOC', 'O', 'O', 'O', 'O...",0
1,"['অর্থনীতি', 'ও', 'আর্থসামাজিক', 'বেশির', 'ভাগ...","['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', '...",0
2,"['নিম্ন', 'আয়ের', 'দেশগুলোকে', 'ছাড়িয়েছে', 'তো...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']",0
3,"['সবাইকে', 'অন্তর্ভুক্ত', 'করে', 'প্রবৃদ্ধি', ...","['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', '...",0
4,"['তবে', 'অন্তর্ভুক্তিমূলক', 'প্রবৃদ্ধি', 'অর্জ...","['O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', '...",0


In [5]:
# take list of list dataset
tokens = df["tokens"].tolist()
tags   = df["tags"].tolist()
print(len(tokens),len(tags)) # taking all the lines here

23889 23889


In [6]:
# tags and tokens fixing
newTags = []
for idx in range(len(tags)):
    demo = tags[idx]
    # getting individual tokens
    demo = demo.replace("[", "" )# at first tabs necessary to add count
    demo = demo.replace("]", "" )# at first tabs necessary to add count
    demo = demo.replace("'", "" )# at first tabs necessary to add count
    demo = demo.replace(" ", "" )# at first tabs necessary to add count
    demo = demo.split(",")
    
    for j in range(len(demo)):
        if demo[j] == "B-PER" or  demo[j] == "I-PER":
            demo[j] = "B-PER"
        else:
            demo[j] = "O"

    newTags.append(demo)
    
tags = newTags
#############
newTokens = []
for idx in range(len(tokens)):
    demo = tokens[idx]
    # getting individual tokens
    demo = demo.replace("[", "" )# at first tabs necessary to add count
    demo = demo.replace("]", "" )# at first tabs necessary to add count
    demo = demo.replace("'", "" )# at first tabs necessary to add count
    demo = demo.replace(" ", "" )# at first tabs necessary to add count
    demo = demo.split(",")

    newTokens.append(demo)
tokens = newTokens
print(len(tokens),len(tags))

23889 23889


In [7]:
print(tokens[0][0])
print(tags[0][0])
# so tags and tokens are fixed

\\ufeffউন্নয়নের
O


In [8]:
print(tokens[0])

['\\\\ufeffউন্নয়নের', 'বিস্ময়', 'বাংলাদেশ', 'বাংলাদেশের', 'অগ্রগতি', 'উদাহরণ', 'দেওয়ার', 'মতোই', '।']


In [9]:
print(tags[0])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [10]:
import itertools
# Flatten the list of lists into a single list
flattened_list = list(itertools.chain(*tags))
# Get the unique elements using set
label_names = list(set(flattened_list))
label_name = label_names
label_names# checking unique tag names

['O', 'B-PER']

In [11]:
for i in range(len(tags)):
    demo =tags[i]
    for j in range(len(demo)):
        demo[j] = label_name.index(demo[j])
    tags[i] = demo

print(tags[:5]) # converting tags to int

[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [12]:
words = tokens[0]
labels = tags[0]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_name[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)
# simulataneous checking of tokens and labels

\\ufeffউন্নয়নের বিস্ময় বাংলাদেশ বাংলাদেশের অগ্রগতি উদাহরণ দেওয়ার মতোই । 
O               O      O        O          O       O      O      O    O 


In [13]:
from transformers import AutoTokenizer

model_checkpoint = "sagorsarker/mbert-bengali-ner"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# initializing classifier

In [14]:
tokenizer.is_fast

True

In [15]:
inputs = tokenizer(tokens[0], is_split_into_words=True)
inputs.tokens()
# using tokenizer to pad suitably # here some words are broke down

['[CLS]',
 '\\',
 '\\',
 'u',
 '##fe',
 '##ff',
 '##উ',
 '##ন',
 '##ন',
 '##যন',
 '##ের',
 'ব',
 '##িস',
 '##ময',
 'বাংলাদেশ',
 'বাংলাদেশের',
 'অ',
 '##গর',
 '##গত',
 '##ি',
 'উ',
 '##দা',
 '##হ',
 '##রণ',
 'দেওযা',
 '##র',
 'মতো',
 '##ই',
 '।',
 '[SEP]']

In [16]:
print(inputs.word_ids()) # this tokenization is useful for bert
# need to understand the in depth theory

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 7, 7, 8, None]


In [17]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

# labelling words and ids with tokens after adding padding tokens

In [18]:
labels = tags[0]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))
# matching with the bert input

[0, 0, 0, 0, 0, 0, 0, 0, 0]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


### creating dataset

In [19]:
# deleting the mismatched data
# reading as tab-separated file format as it would be easier for tf.data.Dataset object
def cleaning_data(tokens, tags):
    mismatch = []
    newtokens = []
    newtags   = []
    flags      = []
    newcount = 0
    for i in range(len(tokens)):
        c = 0
        if len(tokens[i]) > 0 and len(tags[i])>0:
            if len(tokens[i]) != len(tags[i]):
                mismatch.append(i)
            else:
                
                
                
                newtokens.append(tokens[i])
                newtags.append(tags[i])
                
                demo = tags[i]
                for j in demo:
                    if j == 1 or j == 2:
                        c = 1
                if c == 1:
                    flags.append(1)
                else:
                    flags.append(0)
                

                
    print("total number of flag sentences %d"%(sum(flags)))
    print(len(flags),"\n")
    print("total number of mismatched token and tag pair sentneces %d"%(len(mismatch)))
    print("total number of sentneces %d"%(i+1))
    print("final length of dataset %d"%len(newtokens))
    return newtokens,newtags,flags

tokens,tags,flags = cleaning_data(tokens, tags)

total number of flag sentences 6135
23889 

total number of mismatched token and tag pair sentneces 0
total number of sentneces 23889
final length of dataset 23889


In [20]:
# train val test split
from sklearn.model_selection import train_test_split
# Split the data into train, validation, and test datasets
tokens_train, tokens_val, tags_train, tags_val = train_test_split(tokens, tags,stratify=flags, test_size=0.2, random_state=42)
#tokens_val, tokens_test, tags_val, tags_test = train_test_split(tokens_val, tags_val, test_size=0.25, random_state=42)

# Print the split datasets
print("Train dataset:")
print(tokens_train[0])
print(tags_train[0])
print(len(tokens_train),"\n")

print("Validation dataset:")
print(tokens_val[0])
print(tags_val[0])
print(len(tokens_val),"\n")

#print("Test dataset:")
#print(tokens_test[0])
#print(tags_test[0])
#print(len(tokens_test),"\n")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Train dataset:
['ছিল', 'মধ্যেও', 'জয়', 'প্লাজার', 'দুর্ঘটনায়', 'দৃষ্টান্ত', 'দুঃখের', 'মানবতার', 'অনন্য', 'রানা']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
19111 

Validation dataset:
['আইনে', 'বিবাহ', 'তবে', 'হলে', 'নন', 'অনেকে', 'অবগত', 'আছে', 'সে', '<PUNCT>']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4778 



In [21]:
import gc
gc.collect()

90

In [22]:
# converting tokens and tags to datasetdict
from datasets import Dataset, DatasetDict

# Create Dataset objects for train, validation, and test splits
train_dataset = Dataset.from_dict({'tokens': tokens_train, 'tags': tags_train})
validation_dataset = Dataset.from_dict({'tokens': tokens_val, 'tags': tags_val})
#test_dataset = Dataset.from_dict({'tokens': tokens_test, 'tags': tags_test})

# Create a DatasetDict object
raw_datasets = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    #'test': test_dataset
})

In [23]:
raw_datasets["train"].column_names

['tokens', 'tags']

In [24]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [25]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [26]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer, return_tensors="tf"
)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [27]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


<tf.Tensor: shape=(2, 31), dtype=int64, numpy=
array([[-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0, -100],
       [-100,    0,    0,    0,    0,    1,    2,    2,    2,    2,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, -100, -100, -100, -100, -100]])>

In [28]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]
[-100, 0, 0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [29]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=8,#16
)

tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=8,#16
)

### dictionaries

In [30]:
label_names

['O', 'B-PER']

In [31]:
prevnames = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [32]:
id2label = {i: label for i, label in enumerate(prevnames)}
label2id = {v: k for k, v in id2label.items()}

In [33]:
id2label

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC'}

In [34]:
gc.collect()

0

In [35]:
from transformers import TFAutoModelForTokenClassification

# Initialize the model
model = TFAutoModelForTokenClassification.from_pretrained(
    "sagorsarker/mbert-bengali-ner",
    id2label=id2label,
    label2id=label2id,
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForTokenClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


In [36]:
model.config.num_labels

7

In [37]:
#from huggingface_hub import notebook_login

#notebook_login()

In [38]:
from transformers import create_optimizer
import tensorflow as tf

# Train in mixed-precision float16
# Comment this line out if you're using a GPU that will not benefit from this
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_epochs = 2#5#10#
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

In [39]:
tokenizer

BertTokenizerFast(name_or_path='sagorsarker/mbert-bengali-ner', vocab_size=105879, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [40]:
gc.collect()

0

In [41]:
from transformers.keras_callbacks import PushToHubCallback

dirname = "/kaggle/working/mbert-ner-baseline-1"

callback = PushToHubCallback(output_dir=dirname, tokenizer=tokenizer)

model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    callbacks=[callback],
    epochs=num_epochs,
)

Cloning https://huggingface.co/imtiaz114/mbert-ner-baseline-1 into local empty directory.


Download file tf_model.h5:   0%|          | 17.4k/416M [00:00<?, ?B/s]

Download file sentencepiece.bpe.model:   0%|          | 8.00k/4.83M [00:00<?, ?B/s]

Download file tokenizer.json:   1%|1         | 8.74k/754k [00:00<?, ?B/s]

Download file spiece.model:   0%|          | 8.00k/5.38M [00:00<?, ?B/s]

Clean file tokenizer.json:   0%|          | 1.00k/754k [00:00<?, ?B/s]

Clean file sentencepiece.bpe.model:   0%|          | 1.00k/4.83M [00:00<?, ?B/s]

Clean file spiece.model:   0%|          | 1.00k/5.38M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/416M [00:00<?, ?B/s]

Epoch 1/2
2388/2388 [==============================] - 443s 176ms/step - loss: 0.0902 - val_loss: 0.0541
Epoch 2/2
2388/2388 [==============================] - 449s 188ms/step - loss: 0.0466 - val_loss: 0.0426


In [42]:
!pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=9dc0e0afe58e27bd845d9832cc129e19ac17e22a3a391dcc188ac7b088f65327
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [43]:
import evaluate

metric = evaluate.load("seqeval")

### evaluation

In [44]:
import numpy as np

all_predictions = []
all_labels = []
for batch in tf_eval_dataset:
    logits = model.predict_on_batch(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    for prediction, label in zip(predictions, labels):
        for predicted_idx, label_idx in zip(prediction, label):
            if label_idx == -100:
                continue
            #if label_idx > 1:
                #label_idx = 0
            #if predicted_idx > 1:
                #predicted_idx = 0
            all_predictions.append(prevnames[predicted_idx])#label_names
            all_labels.append(prevnames[label_idx])
metric.compute(predictions=[all_predictions], references=[all_labels])

{'PER': {'precision': 0.7642512077294686,
  'recall': 0.8231009365244537,
  'f1': 0.7925851703406814,
  'number': 2883},
 'overall_precision': 0.7642512077294686,
 'overall_recall': 0.8231009365244537,
 'overall_f1': 0.7925851703406814,
 'overall_accuracy': 0.9831450288709345}

# Prediction

In [45]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = dirname#"/kaggle/working/bert-finetuned-ner-baseline-1"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("প্রতিবেদক ছিলেন সাপ্তাহিকের রাজশাহীর বিভাগীয় প্রতিনিধি সায়েম সাবু ।")

Some layers from the model checkpoint at /kaggle/working/mbert-ner-baseline-1 were not used when initializing TFBertForTokenClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForTokenClassification were initialized from the model checkpoint at /kaggle/working/mbert-ner-baseline-1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


[{'entity_group': 'PER',
  'score': 0.9805,
  'word': 'সাযেম',
  'start': 55,
  'end': 60},
 {'entity_group': 'PER',
  'score': 0.9863,
  'word': 'সাব',
  'start': 61,
  'end': 64}]

In [46]:
output = token_classifier("প্রতিবেদক ছিলেন সাপ্তাহিকের রাজশাহীর বিভাগীয় প্রতিনিধি সায়েম সাবু ।")
print(type(output))
output

<class 'list'>


[{'entity_group': 'PER',
  'score': 0.9805,
  'word': 'সাযেম',
  'start': 55,
  'end': 60},
 {'entity_group': 'PER',
  'score': 0.9863,
  'word': 'সাব',
  'start': 61,
  'end': 64}]

In [56]:
person_names = []
current_person = ""

for entity in output:
    if entity['entity_group'] == 'PER':
        current_person += entity['word']
        person_names.append(current_person)
        current_person = ""

    else:
        if current_person:
            current_person = ""

print(person_names)

['সাযেম', 'সাব']


In [57]:
clean_names=[]
for demo in person_names:
    demo = demo.replace("#", "" )
    clean_names.append(demo)
clean_names

['সাযেম', 'সাব']

In [58]:
def gettingPersonName(inputs):
    print(inputs)
    output = token_classifier(inputs)
    
    
    person_names = []
    current_person = ""

    for entity in output:
        if entity['entity_group'] == 'PER':
            current_person += entity['word']
            person_names.append(current_person)
            current_person = ""

        else:
            if current_person:
                #person_names.append(current_person)
                current_person = ""
        
    clean_names=[]
    for demo in person_names:
        demo = demo.replace("#", "" )
        clean_names.append(demo)
    return clean_names

text = "প্রতিবেদক ছিলেন সাপ্তাহিকের রাজশাহীর বিভাগীয় প্রতিনিধি সায়েম সাবু ।"
print(gettingPersonName(text))

প্রতিবেদক ছিলেন সাপ্তাহিকের রাজশাহীর বিভাগীয় প্রতিনিধি সায়েম সাবু ।
['সাযেম', 'সাব']


In [59]:
text = "নিরাপত্তা কর্মকর্তা জান মোহাম্মদ আদালতকে মোশাররফের ‘অসুস্থতা’ সম্পর্কে অবহিত করেন ।"
print(gettingPersonName(text))

নিরাপত্তা কর্মকর্তা জান মোহাম্মদ আদালতকে মোশাররফের ‘অসুস্থতা’ সম্পর্কে অবহিত করেন ।
['জান', 'মোহামমদ', 'মোশাররফের']


In [60]:
text = "এতে রয়েছে কত্থক ‘গীতমালিকা’ শীর্ষক নৃত্য পরিবেশনা।"
print(gettingPersonName(text))

এতে রয়েছে কত্থক ‘গীতমালিকা’ শীর্ষক নৃত্য পরিবেশনা।
[]


In [61]:
text = "ক্রমবর্ধমান রাজনৈতিক উত্তেজনার সঙ্গে মানবতাবিরোধী অপরাধে বিচারের বিষয়টি যুক্ত হয়ে সারা দেশে নিরাপত্তা পরিস্থিতির অবনতি হচ্ছে ।"
print(gettingPersonName(text))

ক্রমবর্ধমান রাজনৈতিক উত্তেজনার সঙ্গে মানবতাবিরোধী অপরাধে বিচারের বিষয়টি যুক্ত হয়ে সারা দেশে নিরাপত্তা পরিস্থিতির অবনতি হচ্ছে ।
[]


In [62]:
text = "প্রেমের ময়দানে ক্যারোলিন ওজনিয়াকি আর ররি ম্যাকইলরয়ের লক্ষণীয় উন্নতিই হয়েছে ।"
print(gettingPersonName(text))

প্রেমের ময়দানে ক্যারোলিন ওজনিয়াকি আর ররি ম্যাকইলরয়ের লক্ষণীয় উন্নতিই হয়েছে ।
['কযারোলিন', 'ওজনিযাকি', 'ররি', 'মযাকইলরযের']


# Downloading folder

In [63]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    """
    zip all the files in a directory
    
    Parameters
    _____
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    _____
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [64]:
zip_dir()

/kaggle/working/directory.zip

# Insight

* Bert is performing too good actually, huge overfitting is occuring, need to tackle it